In [18]:
import pandas as pd
import re
from io import StringIO

path = "/h/kupfersk/cfpr_2026/data_limited_2026/STATSCAN_food_cpi.csv"

# --- 1️⃣ Read full file ---
with open(path, "r", encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

# --- 2️⃣ Keep only lines that begin with a plausible date ---
data_lines = [
    l for l in lines
    if re.match(r"^[A-Za-z]{3}-\d{2}", l.strip())   # e.g., Jan-86
    or re.match(r"^\d{2}-[A-Za-z]{3}", l.strip())   # e.g., 01-Jan
    or re.match(r"^\d{4}-\d{2}", l.strip())         # e.g., 2024-07
]

print(f"✅ Found {len(data_lines)} data rows after filtering")

# --- 3️⃣ Combine & read as CSV ---
data_str = "".join(data_lines)
df = pd.read_csv(StringIO(data_str), header=None)

# --- 4️⃣ Assign column names ---
df.columns = [
    "date",
    "Food",
    "Meat",
    "Fish, seafood and other marine products",
    "Dairy products and eggs",
    "Bakery and cereal products (excluding baby food)",
    "Fruit, fruit preparations and nuts",
    "Vegetables and vegetable preparations",
    "Other food products and non-alcoholic beverages",
    "Food purchased from restaurants"
]

# --- 5️⃣ Parse all date patterns ---
def parse_statcan_date(x):
    x = str(x).strip()
    try:
        if re.match(r"^[A-Za-z]{3}-\d{2}$", x):      # Jan-86
            dt = pd.to_datetime(x, format="%b-%y")
            if dt.year < 1930:
                dt = dt.replace(year=dt.year + 2000)
            return dt
        elif re.match(r"^\d{2}-[A-Za-z]{3}$", x):    # 01-Jan
            dt = pd.to_datetime(x, format="%y-%b")
            if dt.year < 1930:
                dt = dt.replace(year=dt.year + 2000)
            return dt
        elif re.match(r"^\d{4}-\d{2}$", x):          # 2024-07
            return pd.to_datetime(x, format="%Y-%m")
        else:
            return pd.NaT
    except Exception:
        return pd.NaT

df["date"] = df["date"].apply(parse_statcan_date)
df = df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

# --- 6️⃣ Verify ---
print(df.head(3))
print(df.tail(3))
print(f"✅ Date range: {df['date'].min().date()} → {df['date'].max().date()}")
print(f"✅ Rows: {len(df)}")

# --- 7️⃣ Save clean dataset ---
out_path = "/h/kupfersk/cfpr_2026/data_limited_2026/STATSCAN_food_cpi_clean.csv"
df.to_csv(out_path, index=False)
print(f"💾 Clean file saved to: {out_path}")

df


✅ Found 476 data rows after filtering
        date  Food  Meat  Fish, seafood and other marine products  \
0 1986-01-01  67.3  65.1                                     60.6   
1 1986-02-01  66.9  64.2                                     61.3   
2 1986-03-01  67.0  64.2                                     61.3   

   Dairy products and eggs  Bakery and cereal products (excluding baby food)  \
0                     70.9                                              69.3   
1                     70.8                                              70.3   
2                     71.1                                              70.6   

   Fruit, fruit preparations and nuts  Vegetables and vegetable preparations  \
0                                76.0                                   76.0   
1                                77.6                                   68.4   
2                                79.2                                   66.2   

   Other food products and non-alcoholic be

In [28]:
import pandas as pd
import os

# === 1️⃣ Paths ===
base_dir = "/h/kupfersk/cfpr_2026/data_limited_2026"
main_file = os.path.join(base_dir, "STATSCAN_food_cpi_clean.csv")
output_file = os.path.join(base_dir, "CFPR_2026_master_dataset.csv")

# === 2️⃣ Load main CPI dataset ===
main_df = pd.read_csv(main_file, parse_dates=["date"])
main_df = main_df.set_index("date")

print(f"✅ Loaded main CPI dataset: {main_df.shape[0]} rows, {main_df.shape[1]} columns")

# === 3️⃣ Iterate through other CSV files ===
for file in os.listdir(base_dir):
    if not file.endswith(".csv"):
        continue

    # Skip main CPI and output file to avoid duplication
    if file in ["STATSCAN_food_cpi_clean.csv", "CFPR_2026_master_dataset.csv"]:
        continue

    path = os.path.join(base_dir, file)
    col_name = os.path.splitext(file)[0]  # filename without .csv (e.g., 'Climate', 'Geopolitical')

    print(f"🔄 Processing {col_name}")

    df = pd.read_csv(path)

    # --- Find date column ---
    date_col = [c for c in df.columns if "date" in c.lower() or "index" in c.lower()]
    if not date_col:
        print(f"⚠️ Skipping {file} — no date column found.")
        continue

    df = df.rename(columns={date_col[0]: "date"})
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).set_index("date")

    # --- Select numeric columns ---
    numeric_cols = df.select_dtypes(include="number").columns
    if len(numeric_cols) == 0:
        print(f"⚠️ Skipping {file} — no numeric data found.")
        continue

    # --- Handle multiple numeric columns (averaging if needed) ---
    if len(numeric_cols) > 1:
        print(f"ℹ️ {file}: multiple numeric columns ({list(numeric_cols)}). Using their mean.")
        df[col_name] = df[numeric_cols].mean(axis=1)
    else:
        df[col_name] = df[numeric_cols[0]]

    # --- Keep only the single merged column ---
    df = df[[col_name]]

    # --- Merge with main_df on date ---
    main_df = main_df.merge(df, left_index=True, right_index=True, how="left")

# === 4️⃣ Save merged dataset ===
main_df.to_csv(output_file)
print(f"\n✅ Final combined dataset shape: {main_df.shape}")
print(f"✅ Columns: {list(main_df.columns)}")
print(f"💾 Saved to: {output_file}")


✅ Loaded main CPI dataset: 476 rows, 9 columns
🔄 Processing STATSCAN_food_cpi
⚠️ Skipping STATSCAN_food_cpi.csv — no numeric data found.
🔄 Processing Geopolitical_clean
🔄 Processing Climate
🔄 Processing Manufacturing
🔄 Processing Economic

✅ Final combined dataset shape: (476, 13)
✅ Columns: ['Food', 'Meat', 'Fish, seafood and other marine products', 'Dairy products and eggs', 'Bakery and cereal products (excluding baby food)', 'Fruit, fruit preparations and nuts', 'Vegetables and vegetable preparations', 'Other food products and non-alcoholic beverages', 'Food purchased from restaurants', 'Geopolitical_clean', 'Climate', 'Manufacturing', 'Economic']
💾 Saved to: /h/kupfersk/cfpr_2026/data_limited_2026/CFPR_2026_master_dataset.csv


/tmp/ipykernel_36529/2159540280.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


In [29]:
main_df

,Food,Meat,"Fish, seafood and other marine products",Dairy products and eggs,Bakery and cereal products (excluding baby food),"Fruit, fruit preparations and nuts",Vegetables and vegetable preparations,Other food products and non-alcoholic beverages,Food purchased from restaurants,Geopolitical_clean,Climate,Manufacturing,Economic
date,,,,,,,,,,,,,
1986-01-01,67.3,65.1,60.6,70.9,69.3,76.0,76.0,77.5,59.1,NaN,-0.66,99.800,1.4070
1986-02-01,66.9,64.2,61.3,70.8,70.3,77.6,68.4,78.1,59.1,NaN,-0.71,99.200,1.4043
1986-03-01,67.0,64.2,61.3,71.1,70.6,79.2,66.2,78.6,59.3,NaN,-0.42,99.000,1.4009
1986-04-01,67.7,63.6,61.4,71.0,71.3,82.2,71.1,79.5,59.7,NaN,-0.33,98.600,1.3879
1986-05-01,68.2,64.0,61.9,71.4,71.2,83.5,75.3,79.8,59.9,NaN,-0.53,99.300,1.3757
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-01,194.5,219.2,164.8,175.9,197.6,175.8,191.2,184.7,200.1,72733.5,-0.14,267.109,1.3981
2025-05-01,195.4,221.2,166.1,176.3,199.0,177.1,193.4,184.9,200.6,77997.7,-0.13,268.672,1.3867
2025-06-01,195.6,221.5,167.9,177.0,198.0,177.4,186.6,187.5,201.2,84305.3,-0.07,270.395,1.3668
